In [1]:
### Test Cavity Flow
###
### In trying to obtain the proper flow pattern, the neural network has to make a significant 'discovery'
### It wants to learn 0 everywhere except at the top lid, and be like 'can't learn any better than this'
### And so I think the trained result is stochastic -> There is some chance that it will learn in the training session
### The longer the training session, the more likely the NN will end up there

In [2]:
import time
import numpy as np
import math

import torch
import torch.nn as nn
import torch.optim as optim

dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.set_default_dtype(torch.float64)

In [3]:
############## Save model and/or Load model ##############

savemodel = 'CavityFlow'
loadmodel = ''

# Physical Dimension
x_dim = 2
output_dim = 2

# Steady   or Unsteady
# Elliptic or Parabolic
is_unsteady = False
input_dim = x_dim + is_unsteady

################# PDE Coefficients ########################

lidspeed = 1.0
mu = 1.0

def bdry_con_wall(X):
    u = torch.zeros( (X.size(0), output_dim), device=X.device)
    return u

def bdry_con_lid(X):
    u = torch.zeros( (X.size(0), output_dim), device=X.device)

    u[:,0] = lidspeed
    return u

#################  Make the domain  #######################

boundingbox = [ [-1,1], [-1,1] ]

wall_left = {'type':'line',
             'point': [-1,0],
             'normal': [1,0],
             'endpoints': [ [-1,-1], [-1,1] ],
             'boundary_condition': bdry_con_wall }

lid_top = { 'type':'line',
             'point': [0,1],
             'normal': [0,-1],
             'endpoints': [ [-1,1],   [1,1]  ],
             'boundary_condition': bdry_con_lid }

wall_right= {'type':'line',
             'point': [1, 0],
             'normal':  [-1,0],
             'endpoints':  [ [1,-1],   [1,1]  ],
             'boundary_condition': bdry_con_wall }

wall_bot = {'type':'line',
             'point': [0,-1],
             'normal': [0,1],
             'endpoints': [ [-1,-1], [1,-1] ],
             'boundary_condition': bdry_con_wall }

list_of_dirichlet_boundaries = [wall_left, lid_top, wall_right, wall_bot ]
list_of_periodic_boundaries =[]

In [4]:
# Time step
dt = 1e-4

# exit tolerance
tol = 1e-6

# Number of walkers
num_walkers = 2**12
num_ghost = 128
num_batch = 2**12

# Update walkers
# Options: 
#    move -- moves walkers to one of their new locations
#    remake -- remake walkers at each training step
#    fixed -- keeps walkers fixed
update_walkers = 'move'
update_walkers_every = 1

# Number of boundary points 
num_bdry = 2**10

############## Training Parameters #######################

# Training epochs
num_step = 1000
update_print_every = 500

# Neural Network Architecture
nn_depth = 60
nn_width = 4

# Weighting of losses
lambda_bell = 1e-2/dt
lambda_bdry = 1e0

# Learning rate
learning_rate = 1e-2
adam_beta = (0.9,0.999)
weight_decay = 0

In [5]:
class IncompressibleNN(torch.nn.Module):
    
    ### Incompressible neural network
    ### curl operation built in
    
    def __init__(self, depth, width, x_dim, is_unsteady, **nn_param):
        super(IncompressibleNN, self).__init__()
        
        self.x_dim = x_dim
        self.input_dim = self.x_dim + is_unsteady
        
        self.dim_out = 2
        
        modules = []
        modules.append(torch.nn.Linear(self.input_dim, depth))
        for i in range(width - 1):
            modules.append(torch.nn.Linear(depth, depth))
            modules.append(torch.nn.Tanh())
        modules.append(torch.nn.Linear(depth, self.dim_out))
                       
        self.sequential_model = torch.nn.Sequential(*modules)
    
    def curl(self, a, x):
        if self.x_dim == 2:

            e = torch.eye(self.x_dim, device=x.device)
            
            dadx = torch.autograd.grad(a, x, grad_outputs=e[0,:].repeat(a.size(0), 1), 
                                        create_graph=True, retain_graph = True)[0]
            dpdx = torch.autograd.grad(a, x, grad_outputs=e[1,:].repeat(a.size(0), 1),
                                        create_graph=True, retain_graph = True)[0]

            u = torch.stack([dadx[:,1], -dadx[:,0], dpdx[:,0], dpdx[:,1]] , dim=1)
            
        elif self.x_dim == 3:
            e = torch.eye(self.x_dim, device=x.device)

            da0dx = torch.autograd.grad(a, x, grad_outputs=e[0,:].repeat(a.size(0), 1), 
                                        create_graph=True, retain_graph = True)[0]
            da1dx = torch.autograd.grad(a, x, grad_outputs=e[1,:].repeat(a.size(0), 1),
                                        create_graph=True, retain_graph = True)[0]
            da2dx = torch.autograd.grad(a, x, grad_outputs=e[2,:].repeat(a.size(0), 1),
                                        create_graph=True, retain_graph = True)[0]

            u = torch.stack([da2dx[:,1] - da1dx[:,2], da0dx[:,2] - da2dx[:,0], da1dx[:,0] - da0dx[:,1] ], dim=1)         
        return u
    
    def forward(self, x):
        a = self.sequential_model(x)
        u = self.curl(a, x)
            
        return u

In [6]:
### Domain
class Domain:
    ### This class defines the domain using the parameters provided
    ### It sets up the boundingbox and each boundary

    def __init__(self, is_unsteady, boundingbox, 
                list_of_dirichlet_boundaries,
                list_of_periodic_boundaries=False):
        
        self.boundingbox = boundingbox
        self.is_unsteady = is_unsteady

        self.num_of_boundaries = len(list_of_dirichlet_boundaries)
        
        # Unpack dirichlet boundary descriptions
        self.boundaries = []
        for specs in list_of_dirichlet_boundaries:
            ### 2D boundaries
            if specs['type'] == 'line':
                self.boundaries.append( bdry_line( point = specs['point'], 
                                                   normal = specs['normal'],
                                                   endpoints = specs['endpoints'],
                                                   boundary_condition = specs['boundary_condition'] ))
            
            if specs['type'] == 'disk':
                self.boundaries.append( bdry_disk( centre = specs['centre'],
                                                   radius = specs['radius'], 
                                                   boundary_condition = specs['boundary_condition'] ))
          
        # Unpack any periodic boundaries
        self.periodic_boundaries = []
        for specs in list_of_periodic_boundaries:
            self.periodic_boundaries.append( bdry_periodic( variable = specs['variable'],
                                                            base = specs['base'],
                                                            top = specs['top']  ))

### Boundary Classes

# 2D Boundaries
class bdry_disk:
    ### Class structure for a 2D solid disk boundary, the domain being outside the disk
    
    def __init__(self, centre, radius, boundary_condition):
        ### Centre and Radius
        self.centre = torch.tensor( centre )
        self.radius = radius

        self.bdry_cond = boundary_condition
        
        self.angles = [0, 2*math.pi]
            
            
    def make_bdry_pts(self, num_bdry, boundingbox, other_bdrys):
        ### Make random points along the boundary
        
        #theta = 2*math.pi*torch.rand(num_bdry)
        theta = torch.linspace(0,2*math.pi, 2**7)
        
        #rad_theta = torch.cartesian_prod(self.radius*torch.sqrt(torch.linspace(0,1, 2**4)),
        #                                 torch.linspace(0,2*math.pi, 2**6))

        Xbdry = torch.stack((self.radius*torch.cos(theta) + self.centre[0],
                                 self.radius*torch.sin(theta) + self.centre[1]),dim=1 )

        #Xbdry = torch.stack( (rad_theta[:,0]*torch.cos(rad_theta[:,1]) + self.centre[0],
        #                      rad_theta[:,0]*torch.sin(rad_theta[:,1]) + self.centre[1]), dim=1)

        
        #Xbdry = torch.cat( ( Spacebdry, 
        #                     boundingbox[2][1]*torch.rand(num_bdry,1),
        #                     boundingbox[3][1]*torch.rand(num_bdry,1),
        #                     boundingbox[4][1]*torch.rand(num_bdry,1),), dim=1)

    
        ### Check if outside other bdrys
        ### and remake bdry points
        outside = torch.zeros(Xbdry.size(0), dtype=torch.bool)

        for bdry in other_bdrys:
            outside += bdry.dist_to_bdry(Xbdry) < 0
        
        if any(outside):
            Xbdry[outside,:] = self.make_bdry_pts(torch.sum(outside), boundingbox, is_unsteady, other_bdrys)

        return Xbdry
            
    def dist_to_bdry(self, X):
        ### Signed distance to boundary
        ### positive = inside domain
        ### negative = outside domain

        distance = ( torch.norm(X[:,:2] - self.centre.to(X.device),dim=1) - self.radius )
        return distance
    
class bdry_line:
    ### Class structure for a line boundary
    ###       normal vector points inside
    
    def __init__(self, point, normal, endpoints, boundary_condition):
        self.point = torch.tensor(  point )
        self.normal = torch.tensor( normal )
        self.constant = -sum( self.normal*self.point )
        
        self.bdry_cond = boundary_condition
        
        self.endpoints = torch.tensor(endpoints)
        
    def make_bdry_pts(self, num_bdry, boundingbox, other_bdrys):
           
        #Spacebdry = ( self.endpoints[1] - self.endpoints[0] )*torch.rand((num_bdry,1)) + self.endpoints[0]
        Xbdry = ( self.endpoints[1] - self.endpoints[0] )*torch.linspace(0,1, num_bdry)[:,None] + self.endpoints[0]
        
        #Xbdry = torch.cat( ( Spacebdry, 
        #                       boundingbox[2][1]*torch.rand(num_bdry,1),
        #                       boundingbox[3][1]*torch.rand(num_bdry,1),
        #                       boundingbox[4][1]*torch.rand(num_bdry,1),), dim=1)
        
        ### Check if outside other bdrys
        ### and remake bdry points
        #outside = torch.zeros(Xbdry.size(0), dtype=torch.bool)

        #for bdry in other_bdrys:
        #    outside += bdry.dist_to_bdry(Xbdry[:,:2]) < 0
        
        #if any(outside):
        #    Xbdry[outside,:] = self.make_bdry_pts(torch.sum(outside), boundingbox, is_unsteady, other_bdrys)

        return Xbdry
    
    def dist_to_bdry(self, X):
        ### Signed distance to boundary
        ### positive = inside domain
        ### negative = outside domain
        distance = torch.sum( self.normal.to(X.device)*X[:,:2], dim=1) + self.constant
        
        return distance

class Walker_Data(torch.utils.data.Dataset):
    
    def __init__(self, num_walkers, boundingbox, boundaries):
        
        Xold = generate_interior_points(num_walkers, boundingbox, boundaries)
        #region = close_to_region(Xold)
        
        self.location = Xold
        self.num_pts = num_walkers
        #self.region = region
        
    def __len__(self):
        ### How many data points are there?
        return self.num_pts
    
    def __getitem__(self, index):
        ### Gets one sample of data
        ### 
        return self.location[index,:], index
        
class Boundary_Data(torch.utils.data.Dataset):
    
    def __init__(self, num_bdry, boundingbox, boundaries):
        
        Xbdry, Ubdry = generate_boundary_points(num_bdry, boundingbox, boundaries)
        
        self.location = Xbdry
        self.num_pts = Xbdry.size(0)
        self.value = Ubdry
        
    def __len__(self):
        return self.num_pts
    
    def __getitem__(self, index):
        return self.location[index,:], self.value[index,:]    

class Initial_Data(torch.utils.data.Dataset):

    def __init__(self, num_init, boundingbox, boundaries):
        
        Xinit = generate_interior_points(num_init, boundingbox, boundaries)
        Xinit[:,2] = 0

        self.location = Xinit
        self.num_pts = num_init
        self.value = initial_con(Xinit)
        
    def __len__(self):
        return self.num_pts
    
    def __getitem__(self, index):
        return self.location[index,:], self.value[index,:]    

def generate_interior_points(num_walkers, boundingbox, boundaries):
    ### Generate points inside the domain

    X = torch.empty( (num_walkers, len(boundingbox)) )

    for ii in range(len(boundingbox)):
        X[:,ii] = (boundingbox[ii][1] - boundingbox[ii][0])*torch.rand( (num_walkers) ) + boundingbox[ii][0]
        
        #Parameters are given in log scale, rescale back
        #if ii > 2:
        #    X[:,ii] = 10**X[:,ii]

    outside = torch.zeros( X.size(0), dtype=torch.bool)
    for bdry in boundaries:
        outside += bdry.dist_to_bdry(X) < 0
    
    if any(outside):
        X[outside,:] = generate_interior_points(torch.sum(outside), boundingbox, boundaries)
        
    return X

def generate_boundary_points(num_bdry, boundingbox, boundaries):
    ### Generate points along the boundary
    
    points_per_bdry = []
    utrue_per_bdry = []
    
    # Generate num_bdry points for each boundary
    for ii in range(len(boundaries)):
        bdry = boundaries[ii]
        other_bdrys = boundaries[:ii] + boundaries[ii+1:]

        # Generate boundary points
        X_in_bdry =  bdry.make_bdry_pts(num_bdry, boundingbox, other_bdrys)
        U_in_bdry = bdry.bdry_cond(X_in_bdry)

        points_per_bdry.append( X_in_bdry )
        utrue_per_bdry.append( U_in_bdry )

    Xbdry = torch.cat( points_per_bdry, dim=0)
    Ubdry_true = torch.cat( utrue_per_bdry, dim=0)

    # Sample from above boundary points
    #indices = torch.multinomial( torch.arange( len(boundaries)*num_bdry, dtype=torch.float ), num_bdry)
    
    #Xbdry = Xbdry[indices,:]
    #Ubdry_true = Ubdry_true[indices,:]
    
    return Xbdry, Ubdry_true


In [7]:
### Initialization

there_are_boundaries = bool(list_of_dirichlet_boundaries)

nn_param = {'depth': nn_depth,
            'width': nn_width,
            'x_dim': x_dim,
            'is_unsteady': is_unsteady,
            'output_dim': output_dim,
            }
                
################ Preparing the model #################

#print("Initializing the model")

### Make boundaries defining the domain
MyDomain = Domain(is_unsteady, boundingbox, 
                  list_of_dirichlet_boundaries,
                  list_of_periodic_boundaries)

### Initialize the Model
#model = IncompressibleNN(**nn_param) #.to(dev)
import DRLPDE_nn

model = DRLPDE_nn.IncompressibleNN(**nn_param).to(dev)

mseloss = torch.nn.MSELoss(reduction = 'mean')
optimizer = torch.optim.Adam(model.parameters(), 
                        lr=learning_rate, 
                        betas=adam_beta, 
                        weight_decay=weight_decay)

### Create Walkers and Boundary points and Organize into DataLoader
RWalkers = Walker_Data(num_walkers, boundingbox, MyDomain.boundaries)
RWalkers_batch = torch.utils.data.DataLoader(RWalkers, batch_size=num_batch, shuffle=True)

if update_walkers == 'move':
    move_RWalkers = torch.zeros_like(RWalkers.location)

if there_are_boundaries:
    BPoints = Boundary_Data(num_bdry, boundingbox, MyDomain.boundaries)
    BPoints_batch = torch.utils.data.DataLoader(BPoints, batch_size=num_batch, shuffle=True)

if is_unsteady:
    IPoints = Initial_Data(num_init, boundingbox, MyDomain.boundaries)
    IPoints_batch = torch.utils.data.DataLoader(IPoints, batch_size=num_batch, shuffle=True)



In [8]:
def evaluate_NS(X, model, x_dim, boundaries, num_batch, num_ghost, tol):
    
    ### Evaluate model
    Uold = model(X)
    #Uold = UdPold[:,:2]
    #dPold = UdPold[:,2:]

    ### Move walkers
    # Increase dimension of the brownian motion
    #Space_new = X[:,:2].repeat(num_ghost,1) - dt*Uold.detach().repeat(num_ghost,1) + np.sqrt(2*dt*mu)*torch.randn((num_batch*num_ghost, x_dim), device=X.device, requires_grad=True)
    #Time_new = X[:,2,None].repeat(num_ghost,1) - dt
    #Param_new = X[:,x_dim+1:].repeat(num_ghost,1)

    #Xnew = torch.cat( (Space_new, Time_new, Param_new), dim=1 )
    #Xnew = torch.cat( (Space_new, Time_new), dim=1 )

    Xnew = X.repeat(num_ghost,1) - dt*Uold.detach().repeat(num_ghost,1) + np.sqrt(2*dt*mu)*torch.randn((num_batch*num_ghost, x_dim), device=X.device, requires_grad=True)

    Unew = model(Xnew)

    ### Calculate exits and re-evaluate points
    Xnew, Unew, outside = exit_condition(X.repeat(num_ghost,1), Xnew, Unew, boundaries, tol)

    #Unew = UdPnew[:,:2].reshape(num_ghost, num_batch, 2).mean(0).detach()
    #dPnew = UdPnew[:,2:].reshape(num_ghost, num_batch, 2).mean(0).detach()

    Unew = Unew.reshape(num_ghost, num_batch,2).mean(0).detach()

    return Xnew, Uold, Unew, outside[:num_batch]

def exit_condition(Xold, Xnew, Unew, boundaries, tol):
    ### Calculate exit conditions
    outside = torch.zeros( Xnew.size(0), dtype=torch.bool, device=Xnew.device)
    
    for bdry in boundaries:
        outside_bdry = bdry.dist_to_bdry(Xnew) < 0
        if torch.sum(outside_bdry) > 0:
            ### Bisection to get close to exit location up to tolerance tol
            
            Xnew[outside_bdry,:] = find_bdry_exit(Xold[outside_bdry,:], Xnew[outside_bdry,:], bdry, tol)
            #UdPnew[outside_bdry,:2] = bdry.bdry_cond(Xnew[outside_bdry,:])
            Unew[outside_bdry,:2] = bdry.bdry_cond(Xnew[outside_bdry,:])

        outside += outside_bdry

    ### Check for time = 0
    ### Note: This prioritizes time exit over bdry exit
    ### Question: 
    ###     Should we take a point at the initial time (by projecting or something)
    ###     or is within tol good enough?
    #hit_initial = Xnew[:,2] < 0
    #Xnew[hit_initial,:] = find_time_exit(Xold[hit_initial,:], Xnew[hit_initial,:], tol)
    #Unew[hit_initial,:] = initial_con(Xnew[hit_initial,:])

    #outside += hit_initial

    return Xnew, Unew, outside

def find_bdry_exit(Xold, Xnew, bdry, tol):
    ### Bisection algorithm to find the exit between Xnew and Xold up to a tolerance 
    
    Xmid = (Xnew + Xold)/2
    
    dist = bdry.dist_to_bdry(Xmid)
    
    # above tolerance = inside
    # below tolerance = outside
    above_tol = dist > tol
    below_tol = dist < -tol
    
    if torch.sum(above_tol + below_tol) > 0:
        Xnew[below_tol,:] = Xmid[below_tol,:]
        Xold[above_tol,:] = Xmid[above_tol,:]
        
        Xmid[above_tol + below_tol,:] = find_bdry_exit(Xold[above_tol + below_tol,:], Xnew[above_tol + below_tol,:], bdry, tol)

    return Xmid


In [9]:
### Train from Solution

#from scipy.io import loadmat

#pretrain_step = 0
#numplotpts1d = 128
#numplotpts = numplotpts1d**2

#xg = torch.cartesian_prod(torch.linspace(-1.0, 1.0, numplotpts1d, device=dev), 
#                          torch.linspace(-1.0, 1.0, numplotpts1d, device=dev))

#contents = loadmat('tools/Cavity_Flow_2D_Data_128x128.mat')
#u1_true = np.transpose(contents['u'])
#u2_true = np.transpose(contents['v'])
#utrue = torch.tensor( np.concatenate( (u1_true.reshape(numplotpts,1), u2_true.reshape(numplotpts,1)), axis=1), device=dev )


In [10]:
#for step in range(pretrain_step):
#    ug = model(xg.requires_grad_(True))
#    loss = mseloss(ug, utrue)
#    loss.backward()
#    optimizer.step()
#    optimizer.zero_grad()

#torch.save(model.state_dict(), "savedmodels/" + savemodel + ".pt")

In [11]:

################ Training the model #################

#print("Training has begun")

start_time = time.time()

### Run the training loop

for step in range(num_step):

    # Random Walkers - do in batches
    for Xold, index in RWalkers_batch:

        # Send to GPU and set requires grad flag
        Xold = Xold.to(dev).requires_grad_(True)
        #Xold = Xold.requires_grad_(True)

        # Evaluate at old location and Move walkers
        Xnew, Uold, Unew, outside = evaluate_NS(Xold, model, x_dim, MyDomain.boundaries, num_batch, num_ghost, tol)

        # Calculate loss
        #loss = lambda_bell*mseloss(Uold + (dt/2)*dPold, Unew - (dt/2)*dPnew)
        loss = lambda_bell*mseloss(Uold, Unew)
        #print(loss.data.cpu().numpy())

        loss.backward()

        # No importance sampling. Walkers just move
        # If moving walkers save the first ghost walker
        if update_walkers == 'move':
            if any(outside):
                Xnew[:num_batch,:][outside,:] = generate_interior_points(torch.sum(outside),boundingbox,MyDomain.boundaries).to(dev)
            move_RWalkers[index,:] = Xnew[:num_batch].detach().cpu()


    # Boundary Points - do in batches
    for Xbdry, Ubtrue in BPoints_batch:
        Xbdry = Xbdry.to(dev).requires_grad_(True)
        #Xbdry = Xbdry.requires_grad_(True)
        Ubtrue = Ubtrue.to(dev).detach()
        #Ubtrue = Ubtrue.detach()
        Ubdry = model(Xbdry)
        loss = lambda_bdry*mseloss(Ubdry[:,:2], Ubtrue)
        #print(loss.data.cpu().numpy())
        loss.backward()

    # Initial Condition Points - do in batches
    #if is_unsteady:
    #    for Xinit, Uinittrue in IPoints_batch:
    #        Xinit = Xinit.to(dev).requires_grad_(True)
    #        #Xinit = Xinit.requires_grad_(True)
    #        Uinittrue = Uinittrue.to(dev).detach()
    #        #Ubtrue = Uinittrue.detach()
    #        Uinit = model(Xinit)
    #        loss = lambda_init*mseloss(Uinit, Uinittrue)
    #        loss.backward()

    # Make optimization step
    optimizer.step()
    optimizer.zero_grad()

    # Update walkers

    if (step+1) % update_walkers_every == 0:
        if update_walkers == 'move':
            RWalkers.location = move_RWalkers
            RWalkers_Batch = torch.utils.data.DataLoader(RWalkers, batch_size=num_batch, shuffle=True)

        #BPoints.location, BPoints.value = generate_boundary_points(num_bdry, boundingbox, MyDomain.boundaries)
        #BPoints_batch = torch.utils.data.DataLoader(BPoints, batch_size=num_batch, shuffle=True)

        #IPoints.location = generate_interior_points(num_init, boundingbox, MyDomain.boundaries)
        #IPoints.location[:,2] = 0
        #IPoints.value = initial_con(IPoints.location)
        #IPoints_batch = torch.utils.data.DataLoader(IPoints, batch_size=num_batch, shuffle=True)

    # Print statements
    if step == 0:
        print('No errors in first epoch')
        current_time = time.time()
        print('Approx time: {:.0f} minutes'.format((current_time - start_time)*num_step/60))
    if (step+1) % update_print_every == 0:
        current_time = time.time()
        print('step = {0}/{1}, {2:2.3f} s/step, time-to-go:{3:2.0f} min'.format(
                step+1, num_step, (current_time - start_time) / (step + 1), 
            (current_time - start_time) / (step + 1) * (num_step - step - 1)/60))

# Save model as pickle file

torch.save(model.state_dict(), "savedmodels/" + savemodel + ".pt")




No errors in first epoch
Approx time: 45 minutes


KeyboardInterrupt: 